# Sentiment classification example with LSTM

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


**[IMDB movie review sentiment classification dataset](https://faroit.com/keras-docs/0.3.3/datasets/)**

This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

/home/dan/anaconda3/envs/tensor/lib/python3.8/site-packages/keras/datasets/imdb.py:49: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


17465344/17464789 [==============================] - 2s 0us/step


In [3]:
X_train.shape

(25000,)

In [4]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [5]:
X_train.shape

(25000, 500)

In [6]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [7]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


<ipython-input-7-e12b94917f54>:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)
/home/dan/anaconda3/envs/tensor/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 248s 10ms/step - loss: 0.4749 - accuracy: 0.7641 - val_loss: 0.3472 - val_accuracy: 0.8525
Epoch 2/3
25000/25000 [==============================] - 243s 10ms/step - loss: 0.2944 - accuracy: 0.8814 - val_loss: 0.3358 - val_accuracy: 0.8567
Epoch 3/3
25000/25000 [==============================] - 244s 10ms/step - loss: 0.2629 - accuracy: 0.8977 - val_loss: 0.3454 - val_accuracy: 0.8550


In [8]:
# final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.50%


In [9]:
# LSTM with Dropout for sequence classification in the IMDB dataset
from keras.layers import Dropout

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, nb_epoch=3, batch_size=64)

# final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

<ipython-input-9-20a43ad7d13e>:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


<ipython-input-9-20a43ad7d13e>:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X_train, y_train, nb_epoch=3, batch_size=64)


Epoch 1/3
25000/25000 [==============================] - 223s 9ms/step - loss: 0.4697 - accuracy: 0.7677
Epoch 2/3
25000/25000 [==============================] - 222s 9ms/step - loss: 0.3089 - accuracy: 0.8758
Epoch 3/3
25000/25000 [==============================] - 222s 9ms/step - loss: 0.2645 - accuracy: 0.8950
Accuracy: 86.28%


In [10]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))
model.add(LSTM(100, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, nb_epoch=3, batch_size=64)

# final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

<ipython-input-10-ab33a3e10441>:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))
<ipython-input-10-ab33a3e10441>:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, dropout=0.2, recurrent_dropout=0.2)`
  model.add(LSTM(100, dropout_W=0.2, dropout_U=0.2))


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


<ipython-input-10-ab33a3e10441>:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X_train, y_train, nb_epoch=3, batch_size=64)


Epoch 1/3
25000/25000 [==============================] - 254s 10ms/step - loss: 0.5035 - accuracy: 0.7480
Epoch 2/3
25000/25000 [==============================] - 255s 10ms/step - loss: 0.4478 - accuracy: 0.8028
Epoch 3/3
25000/25000 [==============================] - 257s 10ms/step - loss: 0.4493 - accuracy: 0.7969
Accuracy: 83.33%


**1.Exercise - Create a CNN model with a *maxpooling* layer, followed by a LSTM with 100 output dimension. HINT: do not forget that a CNN should have at least one convolutional layer. Check behavior and accuracy.** 

In [11]:
# TODO

In [18]:
from keras.layers import Conv1D, MaxPooling1D

In [24]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))

model.add(Conv1D(64, kernel_size=7, padding="valid", activation="relu", strides=3))
model.add(MaxPooling1D())

model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, nb_epoch=3, batch_size=64)

# final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

<ipython-input-24-316f793827fd>:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))
<ipython-input-24-316f793827fd>:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X_train, y_train, nb_epoch=3, batch_size=64)


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 165, 64)           14400     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 82, 64)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 240,501
Trainable params: 240,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 39s 2ms/step - loss: 0.3989 - accuracy: 0.8033
Epoc